# Machine Learning with Pipelines
### (Incomplete)
### Train, tune, and pickle models using Sweta's cleaned dataset
**Each pipeline will:**
- Standardize data with MinMax (vs standardscaler to accomodate binary dummy variables) 
- Select features with RFECV (recursive feature elimination with cross-validation)
- Tune hyperparameters with GridSearchCV

**Additional:**
- Evaluate performance
- Pickle
- Ensemble for flask predictor tool

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression, HuberRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

#### Pickle Code

In [2]:
# # save the model to disk
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))
 
# # some time later...
 
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

### Load Dummified Data

In [3]:
train = pd.read_csv('dummi data/train_final.csv')
test = pd.read_csv('dummi data/test_final.csv')
# test[test.isna().any(axis=1)]
test = test.dropna(axis=0)

In [4]:
X_train = train.drop(columns=['SalePrice', 'PID', 'Unnamed: 0'])
y_train = train['SalePrice']
X_test = test.drop(columns=['SalePrice', 'PID', 'Unnamed: 0'])
y_test = test['SalePrice']

### Feature Selection

In [5]:
Lasso GridSearchCV
search = GridSearchCV(Lasso(random_state=0, tol=.1), 
                      {'alpha': np.arange(0.01,1000,1)},
                     cv=5,
        scoring='neg_mean_squared_error'
                     )
search.fit(X_train, y_train)
search.best_params_

pickle.dump(search, open('FS_LassoGridSearch.sav', 'wb'))
search = pickle.load(open('FS_LassoGridSearch.sav', 'rb'))
search.best_params_

{'alpha': 98.01}

In [6]:
X_train

,GrLivArea,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_Oth,SaleType_VWD,SaleType_WD,SaleCondition_Normal
0,856,30,7890,6,6,1939,1950,238.0,0.0,618.0,...,0,0,0,0,0,0,0,0,1,1
1,1039,70,8146,4,8,1900,2003,0.0,0.0,405.0,...,0,0,0,0,0,0,0,0,1,1
2,1665,60,8400,8,6,2001,2001,643.0,0.0,167.0,...,0,0,0,0,0,0,0,0,1,1
3,1922,85,7301,7,5,2003,2003,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,1
4,936,20,6000,4,4,1953,1953,0.0,0.0,936.0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,1242,20,17808,4,5,1946,1950,0.0,0.0,484.0,...,0,0,0,0,0,0,0,0,1,1
1834,816,30,8550,6,8,1934,1950,574.0,0.0,242.0,...,0,0,0,0,0,0,0,0,1,1
1835,1724,60,7162,7,5,2003,2004,0.0,0.0,796.0,...,0,0,0,0,0,0,0,0,1,1
1836,1842,60,8826,7,5,2000,2000,841.0,0.0,144.0,...,0,0,0,0,0,0,0,0,1,1


In [9]:
coefficients = search.best_estimator_.coef_
importance = np.abs(coefficients)
features = X_train.columns
keep_feat = np.array(features)[importance > 1000]
keep_feat
# X_train = X_train[keep_feat].drop(columns='MSSubClass')
# X_test = X_test[keep_feat].drop(columns='MSSubClass')

array(['OverallQual', 'OverallCond', 'BsmtFullBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'MSZoning_RL',
       'LandContour_HLS', 'LotConfig_Corner', 'LotConfig_CulDSac',
       'LotConfig_Inside', 'Neighborhood_BrkSide', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_GrnHill',
       'Neighborhood_Mitchel', 'Neighborhood_NAmes',
       'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_SawyerW',
       'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Condition1_Norm',
       'BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex',
       'BldgType_Twnhs', 'HouseStyle_1.5Fin', 'HouseStyle_1Story',
       'HouseStyle_SFoyer', 'HouseStyle_SLvl', 'RoofMatl_CompShg',
       'Exterior1st_BrkFace', 'Exterior1st_CemntBd',
       'Exterior1st_HdBoard', 'Exterior1st_MetalSd',
       'Exterior1st_VinylSd', 'Exterior2nd_Plywood', 'Foundation_BrkTil',
  

In [ ]:
keep_feat.size

In [ ]:
# RFECV and RandomForestRegressor.

In [ ]:
# pickle.dump(rfecv, open('FS_RFECV.sav', 'wb'))

### OLS Pipeline
- HuberRegressor to mitigate impact of outliers
- L2 Regularization

In [ ]:
# hb = HuberRegressor(alpha=1)

# pipe = make_pipeline(
#     # standardize scale
# #     MinMaxScaler(), 
#     HuberRegressor()    
#     # feature selection 
# #     RFECV(hb, cv=4, scoring='accuracy')
# )

# # hyperparameter tuning
# grid_pipe = GridSearchCV(pipe,
#         param_grid={
#             'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#         }, cv=4, refit=True
#     )

# grid_pipe.fit(X_train, y_train)

In [ ]:
# grid_pipe.best_score_

In [ ]:
# hb = HuberRegressor()
# hb.fit(X_train,y_train)

# lm = LinearRegression()
# lm.fit(X_train, y_train)

In [ ]:
ols = make_pipeline(
    MinMaxScaler(),
    LinearRegression()
)

ols.fit(X_train, y_train)

In [ ]:
# evaluate model
y_pred = ols.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mean abs err:', mae)
r2 = r2_score(y_test, y_pred)
print('r2:', r2)

In [ ]:
pickle.dump(ols, open('models/Linear.sav', 'wb'))

### Bayesian Ridge Pipeline

In [ ]:
br = make_pipeline(
    MinMaxScaler(),
    BayesianRidge()
)

br.fit(X_train, y_train)

In [ ]:
# evaluate model
y_pred = br.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mean abs err:', mae)
r2 = r2_score(y_test, y_pred)
print('r2:', r2)

In [ ]:
pickle.dump(br, open('models/BayesianRidge.sav', 'wb'))

### Random Forest Pipeline

In [ ]:
rf = make_pipeline(
    MinMaxScaler(),
    RandomForestRegressor()
)

rf.fit(X_train, y_train)

In [ ]:
# evaluate model
y_pred = rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mean abs err:', mae)
r2 = r2_score(y_test, y_pred)
print('r2:', r2)

In [ ]:
pickle.dump(rf, open('models/RandomForest.sav', 'wb'))

### Gradient Booster Pipeline

In [ ]:
gb = make_pipeline(
    MinMaxScaler(),
    GradientBoostingRegressor()
)

gb.fit(X_train, y_train)

In [ ]:
# evaluate model
y_pred = gb.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mean abs err:', mae)
r2 = r2_score(y_test, y_pred)
print('r2:', r2)

In [ ]:
pickle.dump(gb, open('models/GBooster.sav', 'wb'))

### SVM SVR Pipeline (poor performance)

In [ ]:
svr = make_pipeline(
    MinMaxScaler(),
    SVR()
)

svr.fit(X_train, y_train)

In [ ]:
# evaluate model
y_pred = svr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mean abs err:', mae)
r2 = r2_score(y_test, y_pred)
print('r2:', r2)

In [ ]:
# pickle.dump(pipe, open('SupportVector.sav', 'wb'))

### Sale Range from all Features

In [ ]:
X_test.shape

In [ ]:
# List1 
Name = ['tom', 'krish', 'nick', 'juli'] 
    
# List2 
Age = [25, 30, 26, 22] 
    
# get the list of tuples from two lists. 
# and merge them by using zip(). 
# list_of_tuples = np.array(list(zip(Name, Age))).T
# array = np.array(Age).T
# array

# Assign data to tuples. 
# list_of_tuples  
  

# Converting lists of tuples into 
# pandas Dataframe. 
df = pd.DataFrame({'Name':Name,'Age':Age}) 
     
# # Print data. 
df.pivot_table(columns='Name')

In [ ]:
pretend_features

In [ ]:
def salePrice_range(features):
    pred_list = [ols.predict(features), 
                 br.predict(features), 
                 rf.predict(features),
                 gb.predict(features)]
    low = min(pred_list)
    low_fmt = '${:,.2f}'.format(low[0])
    high = max(pred_list)
    high_fmt = '${:,.2f}'.format(high[0])
    return print(low_fmt,'-',high_fmt) 

pretend_features = X_test.sample(1, random_state=20)

salePrice_range(pretend_features)

In [ ]:
# save test sample for building of flask app
pd.DataFrame(pretend_features).to_csv('sample.csv', index=False)

In [ ]:
sample = pd.read_csv('sample.csv')
str(ols.predict(sample))

### Ensemble (incomplete)

In [ ]:
# combine predictions of weak learners ?
# x_train = np.column_stack((ols_pred, br_pred, rf_pred, gb_pred))

In [ ]:
# def trainStackModel(x_train, y_train, X_test, n_folds, seed):
#     cv = KFold(n_splits=N_folds, random_state=seed)
#     gbm = xgb.XGBRegressor(
#         n_estimators= 600,
#     )

### PCA Pipeline

### Clustering Pipeline

# Sweta model and set sorting

In [ ]:
import pickle 
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score

lasso = pickle.load(open('sweta/lasso.pkl','rb'))
X_train = pd.read_csv('sweta/X_train_age.csv') 
y_train = pd.read_csv('sweta/y_train_age.csv')
X_test = pd.read_csv('sweta/X_test_age.csv')
y_test = pd.read_csv('sweta/y_test_age.csv')

In [ ]:
X_train.columns.values

In [ ]:
train_pred = lasso.predict(X_train)
# test_pred = lasso.predict(X_test)
# mae_train = mean_absolute_error(y_train, train_pred)
# mae_test = mean_absolute_error(y_test, test_pred)
# r2_train = r2_score(y_train, train_pred)
# r2_test = r2_score(y_test, test_pred)
# print('train mae:', mae_train,'\ntest mae:', mae_test)
# print('train r2:', r2_train,'\ntest r2:', r2_test)